In [ ]:
'''

以先前案例組合而成

s3客戶端生成
dynamo客戶端生成
flask app生成

若無S3桶子
    創建S3桶子
若無DynamoDB Tabel
    創建DynamoDB Tabel

第一個方法-
    用戶訪問網站
    看見上傳網站頁面，下方有目前已上傳圖片的表格清單，表格內要有s3的連結名

第二個方法
    用戶上傳圖片
    伺服器收到Request
    生成pre-sign url，上傳的檔案預設需公開
    並插入資料至dynamodb
    讓用戶將圖片傳至pre-sign url
    看見上傳網站頁面，下方有目前已上傳圖片的表格清單，表格內要有s3的連結名

第三個方法
    啟動flask app，使外部port 5000能訪問


'''

In [ ]:
# !pip install boto3 awscli flask requests

In [ ]:
'''

s3客戶端生成
dynamo客戶端生成
flask app生成

'''
import boto3

s3Client = boto3.client('s3', endpoint_url='http://localstack-main:4572')

dynamodb = boto3.resource('dynamodb', endpoint_url='http://localstack:4566')

from flask import Flask

app = Flask(__name__)


In [ ]:
'''

若無S3桶子
    創建S3桶子
若無DynamoDB Tabel
    創建DynamoDB Tabel

'''
response = s3Client.list_buckets()
print(response.get('Buckets'))
if len(response.get('Buckets')) == 0:
    createBucketResult = s3Client.create_bucket(Bucket='cxcxc-aws-certificate')
    print(createBucketResult)

table_list = list(dynamodb.tables.all())
print(table_list)
if list(dynamodb.tables.all()) == []:
    table = dynamodb.create_table(
        TableName='s3-objects-list', 
        KeySchema=[
            {
                'AttributeName': 'image_name',
                'KeyType': 'HASH'  # Partition key
            },
            {
                'AttributeName': 'upload_time',
                'KeyType': 'RANGE'  # Sort key
            }
        ], 
        AttributeDefinitions=[
            {
                'AttributeName': 'image_name',
                'AttributeType': 'S'  # N 代表數字； S 代表字串。
            },
            {
                'AttributeName': 'upload_time',
                'AttributeType': 'S'  
            },
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 5,
            'WriteCapacityUnits': 5
        }
    )
    
    print(table)

table_list = list(dynamodb.tables.all())
table = dynamodb.Table('s3-objects-list')
print(table_list, table.item_count)

In [ ]:
'''

第一個方法-
    用戶訪問網站
    看見上傳網站頁面，下方有目前已上傳圖片的表格清單，表格內要有s3的連結名

'''
import os
from flask import render_template, request, redirect, url_for
from werkzeug.utils import secure_filename
from boto3.dynamodb.conditions import Key, Attr
import datetime
from pprint import pprint


@app.route('/read_images', methods=['GET'])
def read_images():
    
    # 掃描DynamoDB 將所有image 列出
    response = table.scan(
        FilterExpression=Attr('image_name').exists()
    )
    items = response['Items']
    # 準備渲染在頁面上的資料list
    uploaded_images_list = []
    images_url_list = []
    for i in items:
        uploaded_images_list.append(i.get('image_name'))
        images_url_list.append(i.get('image_url'))
        
    return render_template('read_images.html',
                           length = list(range(table.item_count)),
                           uploaded_images = uploaded_images_list, 
                           images_url = images_url_list
                          )
    

In [ ]:
'''
第二個方法
    用戶上傳圖片
    伺服器收到Request
    生成pre-sign url，上傳的檔案預設需公開
    並插入資料至dynamodb
    讓用戶將圖片傳至pre-sign url
    看見上傳網站頁面，下方有目前已上傳圖片的表格清單，表格內要有s3的連結名

'''


UPLOAD_FOLDER = '/home/jovyan/work/story1/upload_images'

@app.route('/upload_images', methods=['GET', 'POST'])
def upload_images():
    if request.method == 'POST':        
        file = request.files.get('file')

        filename = secure_filename(file.filename)
        file_path = os.path.join(UPLOAD_FOLDER, filename)
        # 將照片存檔
        file.save(file_path)
        
        # 上傳照片至S3
        s3Client.upload_file(file_path, 
                             'cxcxc-aws-certificate', 
                             filename)
        # 取得剛上傳照片的presigned url
        presign_url = s3Client.generate_presigned_url(
                                    'get_object',
                                    Params={
                                        'Bucket': 'cxcxc-aws-certificate',
                                        'Key': filename},
                        )
        # 將照片名稱，上傳時間，presigned url 記錄在dynamodb
        table.put_item(
                       Item={
                            "image_name": filename,
                            "upload_time": str(datetime.datetime.utcnow()),
                            "image_url": presign_url
                        }
                    )
    # 掃描DynamoDB 將所有image 列出
    response = table.scan(
        FilterExpression=Attr('image_name').exists()
    )
    items = response['Items']
    # 準備渲染在頁面上的資料list
    uploaded_images_list = []
    images_url_list = []
    for i in items:
        uploaded_images_list.append(i.get('image_name'))
        images_url_list.append(i.get('image_url'))
        
    return render_template('upload_images.html',
                           length = list(range(table.item_count)),
                           uploaded_images = uploaded_images_list, 
                           images_url = images_url_list
                          )
    

In [ ]:
'''
第三個方法
    啟動flask app，使外部port 5000能訪問

'''

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)